<a href="https://colab.research.google.com/github/DynamicLLM/LLM2024/blob/main/src/sample-ai-agent/HurricaneForecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json
import re  # Import the re module for regular expressions

# Class to interact with the Llama model server
class LlamaLLM:
    def __init__(self, url):
        self.url = url
        self.call_count = 0

    def generate(self, prompt):
        self.call_count += 1
        payload = {
            "model": "llama3.2",
            "prompt": prompt,
            "stream": False
        }
        headers = {
            'Content-Type': 'application/json'
        }
        response = requests.post(self.url, headers=headers, data=json.dumps(payload))
        response.raise_for_status()
        return response.json()['response']

# Environment class for hurricane forecasting
class HurricaneEnvironment:
    def __init__(self, hurricane_data):
        self.state = 'initial_state'
        self.goal_achieved = False
        self.result = None
        self.hurricane_data = hurricane_data

    def get_state(self):
        return self.state

    def change_state(self, result):
        self.state = 'goal_state'
        self.result = result

# HurricaneForecastAgent class for hurricane forecasting
class HurricaneForecastAgent:
    def __init__(self, environment, llm, max_attempts=5):
        self.environment = environment
        self.llm = llm
        self.max_attempts = max_attempts

    def perceive(self):
        return self.environment.get_state()

    def reason(self, state):
        prompt = self.create_prompt(self.environment.hurricane_data)
        response = self.llm.generate(prompt).strip()
        print(f"Raw Llama Response: {response}")  # Debugging: print raw response
        decision = self.extract_forecast(response)
        return 'achieve_goal' if decision else 'take_action', decision

    def act(self, action, result):
        if action == 'achieve_goal':
            self.environment.goal_achieved = True
            self.environment.change_state(result)

    def create_prompt(self, hurricane_data):
        prompt = f"""
        The hurricane data is as follows:
        Historical Data: {hurricane_data['historical_data']}
        Current Observations: {hurricane_data['current_observations']}
        Model Predictions: {hurricane_data['model_predictions']}
        Given this information, what is the most likely path and impact of the hurricane?
        """
        return prompt

    def extract_forecast(self, response):
        # Extracts a forecast from the Llama model's response
        # Assuming the response contains a clear path prediction
        path_pattern = re.compile(r'landfall in (.*?)(?:,|\.| )', re.IGNORECASE)
        match = path_pattern.search(response)
        return match.group(1) if match else None

# Example hurricane data
hurricane_data = {
    "historical_data": "Data from previous hurricanes including paths, intensities, and impacts.",
    "current_observations": "Current satellite images, ocean buoy data, and radar data.",
    "model_predictions": "Predictions from various models like the European Center for Medium-Range Weather Forecasts and the National Hurricane Center."
}

# Initialize the Llama model with the server URL
llm = LlamaLLM(url="http://localhost:11434/api/generate")

# Create environment and agent
env = HurricaneEnvironment(hurricane_data)
agent = HurricaneForecastAgent(env, llm)

# Agent perception, reasoning, and action loop
attempts = 0
while not env.goal_achieved and attempts < agent.max_attempts:
    state = agent.perceive()
    action, result = agent.reason(state)
    agent.act(action, result)
    attempts += 1
    print(f"State: {state}, Action: {action}, Result: {result}, Attempts: {attempts}")

if env.goal_achieved:
    print(f"Goal achieved with result: {env.result}")
else:
    print("Failed to achieve goal within the maximum number of attempts.")


Raw Llama Response: Based on the available information, I will provide a hypothetical analysis using the provided categories.

1. Historical Data:

Historical data can provide insights into previous hurricanes that had similar characteristics, such as size, speed, and intensity. By studying these patterns, we can make educated predictions about the potential path and impact of the current hurricane.

2. Current Observations:

Current satellite images, ocean buoy data, and radar data provide real-time information about the storm's current state, including its location, size, shape, and intensity. This data can help us understand the storm's trajectory and predict where it may make landfall.

3. Model Predictions:

The European Center for Medium-Range Weather Forecasts (ECMWF) and the National Hurricane Center (NHC) provide model predictions that forecast the storm's path, intensity, and potential impacts. By combining these models, we can improve our accuracy and confidence in predictin